# Layer Classifier Training

Run this notebook from the project root (`CAD AI`) to build a fitted `layer_clf.pkl`.


In [ ]:
import pathlib, pandas as pd, joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Use the notebook’s working directory to locate your repo
repo_root = pathlib.Path().resolve()
data_path = repo_root / "ml" / "datasets" / "labeled.csv"
assert data_path.exists(), f"{data_path} not found"

df = pd.read_csv(data_path, on_bad_lines="skip").dropna(subset=["Content", "Layer"])

# Pipeline with TF–IDF and a Logistic Regression classifier
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(token_pattern=r"[A-Za-z0-9\\.]+")),
    ("clf", LogisticRegression(max_iter=1000, multi_class="auto"))
])


In [ ]:
# Train/test split and fit
X_train, X_test, y_train, y_test = train_test_split(
    df['Content'], df['Layer'], test_size=0.2, random_state=42
)

pipe.fit(X_train, y_train)
print(classification_report(y_test, pipe.predict(X_test)))

In [ ]:
out_path = repo_root / "ml" / "artifacts" / "layer_clf.pkl"
assert hasattr(pipe.named_steps["tfidf"], "vocabulary_")  # make sure fitted
out_path.parent.mkdir(parents=True, exist_ok=True)  # ensure the directory exists
joblib.dump(pipe, out_path)
print("✓ Model saved to", out_path)